In [ ]:
# ################## DELETE SESSION ##################
import fiftyone as fo
dataset = fo.load_dataset("25_images")
dataset.delete()
# ####################################################

In [1]:
# ################## LIST EXISTING SESSIONS ##################
import fiftyone as fo

datasets = fo.list_datasets()
# dataset = fo.load_dataset("images")
print(fo.list_datasets())

[]


In [ ]:
# ################## CREATE NEW SESSION ##################

import fiftyone as fo
import os

name = "18july"
dataset_dir = "//Aidatastorage/sps/IMAGES_EXPORT_2024JUN26"

ds_17jul = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.FiftyOneDataset,
    name=name,
    progress=True
)

session = fo.launch_app(ds_17jul)

ds_17jul.persistent = True

In [ ]:
# ################## LOAD CURRENT DATASET VIEW ##################
import fiftyone as fo 

ds_17jul = fo.load_dataset('18july')

ds_17jul_session = fo.launch_app(ds_17jul)
ds_17jul_session.wait()

In [ ]:
# ################## EXPORT IMAGES ##################
errors_ids = ds_17jul_session.view

errors_ids.export(
    dataset_type=fo.types.FiftyOneDataset,
    export_dir="C:\\Users\\MaxwellLee\\PycharmProjects\\UVSM\\test",
    abs_paths=True,
)

In [ ]:
# ################## RESIZE IMAGES ##################
import os
from PIL import Image

images_path = r"C:\Users\MaxwellLee\PycharmProjects\UVSM\test\data"

heights = []
widths = []

for filename in os.listdir(images_path):
    if filename.lower().endswith(".jpg"):
        file_path = os.path.join(images_path, filename)
        
        try:
            with Image.open(file_path) as image:
                width, height = image.size
                
                if height >= 640:
                    # Set desired dimensions
                    new_height = 640
                    new_width = 640
                    # Option 1: Maintain aspect ratio
                    #a#spect_ratio = width / height
                    #new_width = int(new_height * aspect_ratio)
                    
                    # Option 2: Set width to a specific value
                    # new_width = 6300
                    # new_height = int(new_width / aspect_ratio)
                    
                    # Resize the image
                    resized_image = image.resize((new_width, new_height), Image.LANCZOS)
                    
                    # Save the resized image
                    resized_image.save(file_path)
                    
                    # Update width and height
                    width, height = resized_image.size
                
                # Append the dimensions to the lists
                heights.append(height)
                widths.append(width)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

if heights:
    print("Minimum height:", min(heights))
else:
    print("Error: No images processed.")